<a class="anchor" id="0"></a>
#RNN con Keras

Ejemplo b&aacute;sico de una RNN con Keras para generar texto.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importar TensorFlow y verificar su versión
import tensorflow as tf
print("Versión de TensorFlow:", tf.__version__)

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import SimpleRNN, Dense, Activation, LSTM
from tensorflow.keras.models import Sequential

def setUpTrainDataFromTxt(INPUT_FILE, SEQLEN, STEP):
    text = ""

    for archivo in INPUT_FILE:
        print("Leyendo archivo ", archivo)
        with open(archivo, 'rb') as fin:
            lines = []
            for line in fin:
                line = line.strip().lower()
                line = line.decode("ascii", "ignore")
                if len(line) == 0:
                    continue
                lines.append(line)
            text = text + " ".join(lines)
        print("cantidad lineas", len(lines))
        print("largo texto", len(text))

    chars = set([c for c in text])
    nb_chars = len(chars)
    char2index = dict((c, i) for i, c in enumerate(chars))
    index2char = dict((i, c) for i, c in enumerate(chars))

    input_chars = []
    label_chars = []
    for i in range(0, len(text) - SEQLEN, STEP):
        input_chars.append(text[i:i + SEQLEN])
        label_chars.append(text[i + SEQLEN])

    X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
    y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
    for i, input_char in enumerate(input_chars):
        for j, ch in enumerate(input_char):
            X[i, j, char2index[ch]] = 1
        y[i, char2index[label_chars[i]]] = 1

    return X, y, nb_chars, input_chars, label_chars, char2index, index2char

INPUT_FILE = [
    "/content/drive/MyDrive/Coordinacion Linea/2022S1/DLY0100/Semana 11/RNN Para generar texto/quijote.txt",
    "/content/drive/MyDrive/Coordinacion Linea/2022S1/DLY0100/Semana 11/RNN Para generar texto/donjuantenorio.txt",
    "/content/drive/MyDrive/Coordinacion Linea/2022S1/DLY0100/Semana 11/RNN Para generar texto/ladivinacomedia.txt"
]

SEQLEN = 10
STEP = 1

X, y, nb_chars, input_chars, label_chars, char2index, index2char = setUpTrainDataFromTxt(INPUT_FILE, SEQLEN, STEP)


# Definir y Entrenar el Modelo RNN

In [ ]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model_rnn = Sequential()
model_rnn.add(
    SimpleRNN(
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True)
    )
model_rnn.add(Dense(nb_chars))
model_rnn.add(Activation("softmax"))

model_rnn.compile(
    loss="categorical_crossentropy",
    optimizer="rmsprop"
    )

for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteración #: %d" % (iteration))

    model_rnn.fit(X, y,
                  batch_size=BATCH_SIZE,
                  epochs=NUM_EPOCHS_PER_ITERATION)

    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    print("Generando texto de la salida: %s" % (test_chars))
    print(test_chars, end="")

    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            Xtest[0, j, char2index[ch]] = 1
        pred = model_rnn.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")

        test_chars = test_chars[1:] + ypred
    print()


# Definir y Entrenar el Modelo LSTM

In [ ]:
model_lstm = Sequential()
model_lstm.add(
    LSTM(
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True)
    )
model_lstm.add(Dense(nb_chars))
model_lstm.add(Activation("softmax"))

model_lstm.compile(
    loss="categorical_crossentropy",
    optimizer="rmsprop"
    )

for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteración #: %d" % (iteration))

    model_lstm.fit(X, y,
                   batch_size=BATCH_SIZE,
                   epochs=NUM_EPOCHS_PER_ITERATION)

    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    print("Generando texto de la salida: %s" % (test_chars))
    print(test_chars, end="")

    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            Xtest[0, j, char2index[ch]] = 1
        pred = model_lstm.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")

        test_chars = test_chars[1:] + ypred
    print()


In [ ]:
# Modelo. SingleRNN con una capa fully connected para calcular la predicción

HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(
    SimpleRNN(
        HIDDEN_SIZE,
        return_sequences=False,
                                        #-> Numero de caracteres distintos en el texto
        input_shape=(SEQLEN, nb_chars), #-> Las cadenas de entrenamiento son de 10
        unroll=True)
    )
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(
    loss="categorical_crossentropy",
    optimizer="rmsprop"
    )

In [ ]:
# Entrenamos el modelo en n iteraciones y probamos la salida en cada iteración

for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteración #: %d" % (iteration))

    #Entrenamiento de la iteración
    model.fit(X, y,
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS_PER_ITERATION)

    # test de la iteración
    #en base a una semilla (texto inicial), generamos una salida
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    print("Generando texto de la salida: %s" % (test_chars))
    print(test_chars, end="")

    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for i, ch in enumerate(test_chars):   #-> Codificamos los caracteres
            Xtest[0, i, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)] #-> Decodificamos la predicción
        print(ypred, end="")

        test_chars = test_chars[1:] + ypred
    print()
